In [106]:
import numpy as np
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from sklearn.feature_extraction.image import extract_patches_2d
from os import listdir
from skimage import io, transform, util, img_as_float
from tqdm import tqdm_notebook
import warnings
from keras_tqdm import TQDMNotebookCallback
from PIL import Image
import numpy as np
import pandas as pd
from fastnumbers import fast_real
%matplotlib inline

In [107]:
def getLinesFromFile (path):
    data = []
    f = open(path, 'r', encoding='utf-8')
    line = f.readline()
    while line:
        line = f.readline()
        if line.split("	")[0] != "" :
            num1 = line.split("	")[0]
            num2 = line.split("	")[1].split("\n")[0]
            data.append(fast_real(num1))
            data.append(fast_real(num2))
    f.close()
    return data

In [108]:
def getTextAndBackAreas (yLinesCoords):
    res = np.zeros(3508)
    i = 0
    while i < len(yLinesCoords)/2:
        j = yLinesCoords[2*i]
        while j < yLinesCoords[2*i + 1]:
            res[j] = 1
            j += 1
        i += 1   
    return res

In [109]:
def convertJpg2Png (path):
    im = Image.open(path)
    im.save(path.split('.')[0] + '.png')
    pass

In [110]:
def convert2greyscale (path):
    img = Image.open(path).convert('LA')
    img.save(path.split('.')[0] + '_greyscale.png')
    pass

In [111]:
def resizePng (path, newWidth, newHeigth):
    img = Image.open(path)
    resized_img = img.resize((newWidth, newHeigth), Image.ANTIALIAS)
    resized_img.save(path.split('.')[0] + '_resized.png')
    pass

In [112]:
def agregate2array (path, width, heigth):
    img = Image.open(path)
    pix = np.array(img)
    res = []
    for i in range(heigth):
        greySum = 0
        for j in range(width):
            greySum += pix[i][j][0]/pix[i][j][1]
        greyAverage = greySum/width
        res.append(greyAverage)
    return np.array(res)

In [113]:
path_to_lines = "582.txt"
path_to_img = "582.jpg"
width = 2480
heigth = 3508

In [114]:
yLinesCoords = getLinesFromFile (path_to_lines)
oneZeroArr = getTextAndBackAreas(yLinesCoords)

convertJpg2Png (path_to_img)
convert2greyscale (path_to_img)
greyAgrArr = agregate2array (path_to_img.split('.')[0] + '_greyscale.png', width, heigth)

In [115]:
print (oneZeroArr.shape)

(3508,)


In [116]:
print (greyAgrArr.shape)

(3508,)


In [119]:
oneZeroArr = oneZeroArr.reshape (1, -1)
greyAgrArr = greyAgrArr.reshape (1, -1)

In [120]:
print (oneZeroArr.shape)
print (greyAgrArr.shape)

(1, 3508)
(1, 3508)


In [129]:
path_to_img2 = "583.jpg"
convertJpg2Png (path_to_img2)
convert2greyscale (path_to_img2)
greyAgrArr2 = agregate2array (path_to_img2.split('.')[0] + '_greyscale.png', width, heigth)
greyAgrArr2 = greyAgrArr2.reshape(1, -1)

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Activation

max_features = 1000
maxlen = 3508 
batch_size = 1

model = Sequential()
model.add(Embedding(max_features, 3508, input_length=maxlen))
model.add(LSTM(3508))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam'
              )

model.fit(
    greyAgrArr, oneZeroArr, 
    batch_size=batch_size, 
    nb_epoch=1,
    verbose = 1,
    
)

result = model.predict_proba(greyAgrArr2)
print(result)

/home/ivan/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
